# Import libraries

In [1]:
import os
import sys

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import numpy as np
import pandas as pd  # Import Pandas and Numpy to create databases
import tensorflow as tf  # Import tensorflow to do tensor manipulations

from abyss.glob import glob
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
phenotypes_labels = {
    "bc": "Breast Cancer",
    "cad": "Coronary Artery Disease",
    "ms": "Multiple Sclerosis",
    "t2d": "Type 2 Diabetes",
    "pc": "Prostate Cancer",
    "ibd": "Inflammatory Bowel Disease"
}

scenario_labels = {
    "no_cov": "no covariates",
    "abyss": "Abyss correction",
    "cov": "PCA correction",
    "abyss_cov": "Abyss + PCA as covariates"
}
def rename_duplicates(df):
    cols = pd.Series(df.columns)
    for dup in cols[cols.duplicated()].unique():
        print(dup)
        cols[cols[cols == dup].index.values.tolist()] = [dup + '_' + str(i) if i != 0 else dup for i in range(sum(cols == dup))]
    df.columns = cols
    return df

In [18]:
pheno_name = "bc"

In [19]:
path_pheno = f"../data_ignored/UKBiobank/phenotype"
path_geno = f"../data_ignored/UKBiobank/genotype"

In [20]:
pheno = pd.read_pickle(f"{path_pheno}/{pheno_name}.pkl")

In [21]:
pheno

,FID,IID,phenotype
0,5568535,5568535,1
1,3655942,3655942,1
2,5439742,5439742,0
3,2880492,2880492,0
4,2164377,2164377,1
...,...,...,...
12913,1069957,1069957,1
12914,4188122,4188122,0
12915,5894824,5894824,0
12916,3829752,3829752,0
